In [ ]:
from lib import dataset_manager, exploratory_analysis, hypothesis_tests, linear_regression

# FERRAMENTAS DE DEBUGGING - REMOVER ANTES DE ENTREGAR

In [ ]:
%load_ext autoreload
%autoreload 2

# Ficha técnica

## Alunos

### Francesco Brandão Ferreira de Carli

Matrícula: 2018058392

Email: francescobfc@ufmg.br

### Guilherme de Oliveira Silva

Matrícula: 2018058490

Email: guilhermeoliveiras@ufmg.br

## Tema
Análise municipal dos impactos de infraestrutura de Internet na taxa de aprovação de alunos de escolas públicas

## Apresentação
(Link para o vídeo)

# Introdução

intro

In [ ]:
dataset = dataset_manager.get_working_dataset()
dataset.head()

# Análise exploratória

## Dados de acesso à Internet

### Visão geral

As variáveis a seguir foram coletadas a partir do mapa do [SIMET](https://simet.nic.br/mapas-app.html). A plataforma disponibiliza estatísticas de testes de velocidade de acesso à Internet para uma pequena região e categoriza os resultados em:

1. operadora do serviço
2. faixa de velocidade, em Mb/s
3. tipo de conexão: `mobile` (redes móveis) ou `broadband` (banda larga fixa)

De posse dos dados, agregamos as estatísticas coletadas por município e tipo de conexão, contabilizando o número de operadoras únicas e o total de dispositivos por faixa. Para as variáveis de dispositivos por faixa, ainda normalizamos os resultados pela população, a fim de tornar as regiões comparáveis entre si – isto é, a variável está na unidade *dispositivos na faixa n por habitante*. Assim, a variável `type` é categórica, enquanto que as variáveis `tcp_range_<n>` (`<n>` representa a faixa de velocidade) e `num_operadoras` são numéricas – respectivamente, contínuas e discreta.

In [ ]:
exploratory_analysis.get_dataset_description(dataset, 'internet')

### Conclusões - Internet

Nesta seção, observamos que a distribuição de dispositivos por tipo e faixa de velocidade evidenciam que a banda larga fixa é, em média, mais rápida que as redes móveis; e que a maioria dos dispositivos observados nas faixas mais baixas de velocidade é do tipo rede móvel. Isto corrobora as estimativas publicadas pelas plataformas [Statista](https://www.statista.com/statistics/1135124/brazil-internet-speed/) e [Speedtest](https://www.speedtest.net/global-index/brazil), que afirmam que as redes fixas de fato apresentam desempenho médio superior às redes móveis. Geograficamente, os acessos de velocidade superior estão concentrados nas regiões Sul e Sudeste; esta última região, inclusive, possui aproximadamente o triplo de dispositivos por habitante nas faixas superiores de velocidade (15 a 25, 25+ Mbps) se comparada às regiões Norte e Nordeste.

Além disso, estimamos a contagem de operadoras únicas de Internet por região como *proxy* para a infraestrutura de comunicações disponível em cada município. Ao agregarmos em nível regional, novamente nota-se que, caracteristicamente, municípios da região Sudeste possuem mais opções de prestadores de serviço do que municípios de outras regiões – em especial da Norte; entre as demais, as diferenças são menos significativas. Assim, os dados apontam para a possibilidade de que a região Norte esteja aquém das demais em termos de infraestrutura. De fato, considerados os desafios geográficos fisicamente impostos pela região, este seria o resultado esperado.

**Os gráficos utilizados na análise estão disponíveis no [Apêndice B.1](#1.-Dados-de-Internet)**.

## Dados de educação

### Visão geral

As colunas do conjunto de variáveis de educação são todas numéricas e contínuas. Aqui, cada sufixo indica o nível de educação considerado:

| Sufixo  | Nível de educação     | Anos escolares     |
| :------ | --------------------- | ------------------ |
| `_ef_1` | Ensino Fundamental I  | 1º ao 5º ano       |
| `_ef_2` | Ensino Fundamental II | 6º ao 9º ano       |
| `_em`   | Ensino Médio          | 1ª, 2ª e 3ª séries |

**Taxa de aprovação (Ens. Fund. I, Ens. Fund. II, Ens. Médio):** observações em pontos percentuais, com uma casa decimal, no intervalo [0.0, 100.0].

**SAEB:** observações representam a nota média, na [escala SAEB](https://academia.qedu.org.br/prova-brasil/454-2/), que varia de acordo com a disciplina e nível de ensino. A tabela abaixo resume as faixas de valores possíveis.

| Nível de ensino       | Língua Portuguesa | Matemática |
| :-------------------- | ----------------- | ---------- |
| Ensino Fundamental I  | 0 a 350           | 125 a 350  |
| Ensino Fundamental II | 200 a 400         | 200 a 425  |
| Ensino Médio          | 225 a 425         | 225 a 475  |

**Ideb:** observações referem-se ao Índice de Desenvolvimento da Educação Básica e admitem valores entre 0 e 10, com uma casa decimal, para todos os níveis de educação.

In [ ]:
exploratory_analysis.get_dataset_description(dataset, 'education')

### Conclusões - Educação

À exceção do indicador de Taxas de Aprovação, as variáveis observadas tem formato aproximadamente de sino aparentemente bem distribuída ao redor da média – linha pontilhada em vermelho. 

Como um todo, observamos que:

1. Os indicadores *deterioram-se* à medida em que se avança pelos níveis de educação

A concentração de pontos desloca-se à esquerda do eixo x entre o Ensino Fundamental I e Ensino Médio.

2. Ao observamos as caudas da distribuição por região – isto é, a contagem de escolas por região cujo desempenho seja superior a 90% ou inferior a 10% das demais do país –, notamos que as escolas de maior desempenho estão fortemente concentradas no eixo sul-sudeste, e as de menor desempenho, no eixo norte-nordeste.    

Em alguns indicadores, a região norte sequer possui cidades de média suficientemente grande para que se enquadre no decil superior. Isto acontece mais frequentemente ao observarmos os dados referentes ao ensino básico.

**Os gráficos utilizados na análise estão disponíveis no [Apêndice B.2](#2.-Dados-de-Educa%C3%A7%C3%A3o)**.

## Indicadores sociais

### Visão geral

As colunas do conjunto de indicadores sociais são todas numéricas e contínuas. Suas unidades de medida são

| Variável  | Unidade de medida     | Observações     |
| :-------- | --------------------- | ------------------ |
| `populacao` | Número de indivíduos | Dados do Censo de 2010 |
| `exp_vida`  | Média de anos | Esperança de vida ao nascer |
| `analfabetismo` | Analfabetos por 1000 habitantes | Pessoas que de 15 anos ou mais que não sabem ler ou escrever ao menos um bilhete simples |
| `tx_pobreza` | Percentual da população | Pessoas vivendo com menos de USD 5,5 PPC (paridade de poder de compra) por mês |
| `pib_per_capita` | Milhares de BRL por habitante | Em BRL (2016) |
| `mortalidade_inf` | Mortalidade de crianças por 1000 habitantes | Considera crianças que morreram antes de completar 1 ano de vida | 
| `maternidade_inf` | Percentual de meninas que tiveram filhos | Meninas de 10 a 14 anos de idade |

### Conclusões - Indicadores sociais

Os indicadores sociais apresentados servirão de variáveis de controle para a regressão. Antes que apresentemos qualificações sobre as regiões, é necessário frisar que os resultados apresentam certo viés que favorece as regiões Sul e Sudeste: como consideramos apenas as cidades que possuam ao menos 1 registro de teste de velocidade – e dada a concentração da infraestrutura de Internet nessa região –, existirão mais observações para cidades pertencentes a este eixo. Assim, os conclusões devem ser interpretadas com cautela.

Em termos de indicadores, nota-se que o Sudeste, além de concentrar a maior população considerada, também concentra os melhores resultados em todos os indicadores considerados, e o Sul tem desempenho similar. As regiões Norte e Nordeste, por outro lado, encontram-se na situação oposta: seus resultados são todos inferiores às regiões anteriores. A isto certamente está associada a distribuição de riqueza per capita no país, que está diretamente correlacionada ao bom desempenho nestes indicadores. Como caso atípico ao padrão constatado, temos apenas a expectativa de vida: apesar de todas as disparidades pontuadas, a esperança de vida média é muito similar entre todas as regiões – e a amplitude de observações é menor que a das demais variáveis.

**Os gráficos utilizados na análise estão disponíveis no [Apêndice B.3](#3.-Indicadores-Sociais)**.

### Correlações entre Internet e Educação

Partimos da hipótese de que a maior prevalência de dispositivos em faixas superiores de velocidade de acesso à Internet contribui positivamente para indicadores educacionais, em especial aos que se referem ao Ensino Médio – dada a maior autonomia exigida dos estudantes deste nível. Assim, analisamos a matriz de correlações e constatamos que a relação entre variáveis de acesso à Internet e indicadores educacionais é muito pouco significativa – não há correlação absoluta que supere o patamar de **0.1** entre variáveis destes grupos. De fato, as correlações mais significativas são observadas apenas em variáveis intragrupais, conforme esperado.

A seguir, respectivamente, a matriz de correlações e sua representação gráfica.

In [ ]:
exploratory_analysis.get_internet_education_corr(dataset)

# Testes de hipótese

## Diferenças entre taxas de aprovação por região

Conduzimos uma série de Testes A/B a fim de determinar se há diferença estatística significativa entre taxas de aprovação – consideradas todos os níveis de educação – de escolas localizadas na região Sudeste se comparadas às escolas das demais regiões. O teste consistiu nas hipóteses

$
\begin{cases}
H_0: taxa\ de\ aprovacao\ media\ (Sudeste)\ = taxa\ de\ aprovacao\ media\ (Regiao_i)\\
H_1: taxa\ de\ aprovacao\ media\ (Sudeste)\ > taxa\ de\ aprovacao\ media\ (Regiao_i)
\end{cases}\\
Para\ i\ \in \{Norte, Nordeste, Sul, Centro-Oeste\}
$

Com base nos intervalos de confiança gerados, pode-se afirmar que

* Existe evidência estatística de que as taxas de aprovação do Ensino Fundamental I e Ensino Médio do Sudeste são maiores do que as mesmas taxas de todas as demais regiões
* Para o Ensino Fundamental II, há evidência significativa de que as taxas de aprovação de escolas do Sudeste superem todas as demais regiões, **exceto o Centro-Oeste**

Portanto, rejeitamos a hipótese nula para todos os níveis e todas as regiões de comparação, exceto para a comparação de Ensino Fundamental II entre Sudeste e Centro-Oeste.

**Os intervalos de confiança para este teste estão disponíveis no [Apêndice C.1](#1.-Estat%C3%ADstica-do-Teste-A/B:-diferen%C3%A7as-entre-taxas-de-aprova%C3%A7%C3%A3o-por-estado)**.

## Diferenças entre taxas de aprovação de municípios pobres e ricos

Conduzimos um segundo teste de hipóteses para averiguar se há evidência estatística suficiente para afirmar que municípios ricos possuem, em média, taxas de aprovação superiores aos municípios pobres. O teste tem a forma

$
\begin{cases}
H_0: taxa\ de\ aprovacao\ media\ (municipios\ ricos)\ = taxa\ de\ aprovacao\ media\ (municipios\ pobres)\\
H_1: taxa\ de\ aprovacao\ media\ (municipios\ ricos)\ > taxa\ de\ aprovacao\ media\ (municipios\ pobres)
\end{cases}
$

Para classificar municípios entre ricos ou pobres, consideramos como **ricos** todos aqueles cujo PIB per capita fossem maior que a mediana do dataset, e o contrário para municípios **pobres**. Novamente, conduzimos testes individuais para cada nível de educação. Desta vez, empregamos a metodologia Bootstrap para gerar os intervalos de confiança com base em diferenças, e **os gráficos elaborados estão disponíveis no [Apêndice C.2](#2.-Estat%C3%ADstica-do-Teste-A/B:-diferen%C3%A7as-entre-taxas-de-aprova%C3%A7%C3%A3o-entre-munic%C3%ADpios-pobres-e-ricos)**.

Os testes apontam que

* Para os níveis Ensino Fundamental I e II, há diferença significativa entre as médias de aprovação de municípios ricos e pobres – embora esta diferença seja de menor magnitude para o Ensino Fundamental II
* Para o Ensino Médio, a diferença entre as médias não é suficiente para afirmar que sejam estatisticamente diferentes.

Assim, rejeitamos a hipótese nula para Ensino Fundamental I e II, e não rejeitamos para Ensino Médio.

## Diferenças entre taxas de aprovação entre municípios com menor e maior acesso à Internet de alta velocidade

O terceiro e último teste de hipóteses implementado verifica se há evidências suficientes para afirmar que municípios com maior acesso a redes de Internet de alta velocidade possuem maiores taxas médias de aprovação para os três níveis de educação avaliados. A contagem total de dispositivos por habitante nas faixas iguais ou superiores a 10 Mb/s foi tomada como *proxy* para acesso a redes de alta velocidade. Novamente, classificamos municípios entre um grupo ou outro comparando esta contagem à mediana da população considerada. **Ademais, assim como no teste anterior, foi conduzido via Bootstrap de diferenças, e a representação gráfica de seus resultados está disponível no [Apêndice C.3](#3.-Estat%C3%ADstica-do-Teste-A/B:-diferen%C3%A7as-entre-taxas-de-aprova%C3%A7%C3%A3o-entre-munic%C3%ADpios-com-maior-e-menor-acesso-%C3%A0-Internet-de-alta-velocidade)**. 

O teste compara as hipóteses 

$
\begin{cases}
H_0: taxa\ de\ aprovacao\ media\ (municipios\ com\ maior\ acesso\ a\ Internet\ de\ alta\ velocidade)\ = taxa\ de\ aprovacao\ media\ (municipios\ com\ menor\ acesso\ a\ Internet\ de\ alta\ velocidade)\\
H_1: taxa\ de\ aprovacao\ media\ (municipios\ com\ maior\ acesso\ a\ Internet\ de\ alta\ velocidade)\ > taxa\ de\ aprovacao\ media\ (municipios\ com\ menor\ acesso\ a\ Internet\ de\ alta\ velocidade)
\end{cases}
$

Os resultados colhidos mostram que há evidência estatística suficiente para afirmar que as taxas de aprovação no Ensino Fundamental I e II são maiores em municípios com maior acesso a redes de alta velocidade, mas não podemos afirmar o mesmo para as taxas de aprovação do Ensino Médio. Portanto rejeitamos a hipótese nula para Ensino Fundamental I e II, mas não rejeitamos para Ensino Médio.

# Regressões

## Regressões univariadas

Implementamos três modelos – um para cada nível de ensino – para avaliar como a quantidade de dispositivos (por habitante) na menor faixa de velocidade de conexão (0 a 2 Mb/s) está correlacionada com suas respectivas taxas de aprovação de um município. A escolha desta faixa de velocidade como variável independente foi motivada pelo fato de ser a faixa com maior presença entre os municípios que compõem a base de análise.

### Ensino Fundamental I

Os resultados da regressão, apresentados a seguir, indicam a existência de correlação positiva entre dispositivos nesta faixa de velocidade de Internet e taxa de aprovação de estudantes do Ensino Fundamental I. De fato, o p-valor é muito próximo de zero, e, assim, a correlação é estatisticamente significativa.

O Coeficiente de Determinação, porém, é de apenas 0.003. Portanto, ainda que a correlação seja estatisticamente significativa, seu poder explicativo é bastante pequeno.

Finalmente, o *plot* da distribuição de resíduos denota que estes nada se assemelham a uma distribuição normal – um forte indício de que uma regressão linear não seja adequada para modelar esta relação.

**As representações gráficas da regressão e dos resíduos estão disponíveis no [Apêndice D.1.I](#I.-Dispositivos/hab-na-faixa-0-a-2-Mb/s-e-taxa-de-aprova%C3%A7%C3%A3o-no-Ensino-Fundamental-I)**.

In [37]:
linear_regression.get_univariate_regression(dataset, 'ef_1_internet')

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          tx_aprov_ef_1   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     13.80
Date:                Fri, 19 Mar 2021   Prob (F-statistic):           0.000205
Time:                        20:45:16   Log-Likelihood:                -15705.
No. Observations:                4927   AIC:                         3.141e+04
Df Residuals:                    4925   BIC:                         3.143e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         94.3538      0.099    949.693      0.000      94.159      94.549
0_a_2_mbs    225.7516     60.768      3.715      0.000     106.618     344.885
==============================================================================
Omnibus:                     1352.810   Durbin-Watson:                   0.794
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3442.610
Skew:                          -1.493   Prob(JB):                         0.00
Kurtosis:                       5.803   Cond. No.                         727.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Ensino Fundamental II

O cálculo de regressão estimou um coeficiente negativo para a correlação entre a quantidade de dispositivos/habitante com acesso entre 0 e 2 Mb/s e taxa de aprovação no Ensino Fundamental II. Porém, o p-valor deste regressor é maior que o limiar de 5% de significância e, portanto, **não há correlação estatisticamente entre as variáveis consideradas**. Somados a isso o Coeficiente de Determinação igual a zero, a inclinação da reta de regressão aproximadamente nula e distribuição não-normal dos resíduos, pode-se afirmar que a regressão linear não é adequada para modelar esta relação.

**A representação gráfica da regressão está disponível no [Apêndice D.1.II](#II.-Dispositivos/hab-na-faixa-0-a-2-Mb/s-e-taxa-de-aprova%C3%A7%C3%A3o-no-Ensino-Fundamental-II)**.

In [42]:
linear_regression.get_univariate_regression(dataset, 'ef_2_internet')

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          tx_aprov_ef_2   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     2.118
Date:                Fri, 19 Mar 2021   Prob (F-statistic):              0.146
Time:                        21:12:47   Log-Likelihood:                -15788.
No. Observations:                4941   AIC:                         3.158e+04
Df Residuals:                    4939   BIC:                         3.159e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         91.9823      0.100    919.839      0.000      91.786      92.178
0_a_2_mbs    -88.9916     61.143     -1.455      0.146    -208.860      30.876
==============================================================================
Omnibus:                     1091.612   Durbin-Watson:                   1.087
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2701.980
Skew:                          -1.210   Prob(JB):                         0.00
Kurtosis:                       5.696   Cond. No.                         727.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Ensino Médio

Finalmente, estimamos a correlação entre taxa de aprovação dos estudantes de Ensino Médio de um município e a quantidade *per capita* de dispositivos com acesso à Internet de até 2 Mb/s. Para esta especificação do modelo, temos um coeficiente negativo para a variável independente estatisticamente significativa: o p-valor está abaixo do limiar de significância e é aproximadamente igual a zero. O Coeficiente de Determinação de 0.003 demonstra o pequeno poder explicativo da regressão.

Entretanto, ainda que seus resíduos estejam melhor distribuídos, ainda nada se aproximam de uma distribuição normal – constatando, mais uma vez, que a abordagem linear univariada não é suficiente para modelar esta relação.

**A representação gráfica está disponível no [Apêndice D.1.III](#III.-Dispositivos/hab-na-faixa-0-a-2-Mb/s-e-taxa-de-aprova%C3%A7%C3%A3o-no-Ensino-M%C3%A9dio)**.

In [43]:
linear_regression.get_univariate_regression(dataset, 'em_internet')

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            tx_aprov_em   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     15.37
Date:                Fri, 19 Mar 2021   Prob (F-statistic):           8.96e-05
Time:                        21:13:18   Log-Likelihood:                -17654.
No. Observations:                4937   AIC:                         3.531e+04
Df Residuals:                    4935   BIC:                         3.533e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         86.2190      0.146    589.213      0.000      85.932      86.506
0_a_2_mbs   -351.0896     89.552     -3.920      0.000    -526.652    -175.527
==============================================================================
Omnibus:                      299.789   Durbin-Watson:                   0.798
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              356.475
Skew:                          -0.657   Prob(JB):                     3.91e-78
Kurtosis:                       3.075   Cond. No.                         728.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Regressões multivariadas

A seção de [correlações entre as variáveis que compõem o dataset](#Correla%C3%A7%C3%B5es-entre-Internet-e-Educa%C3%A7%C3%A3o) sugere que a adição de um vetor de variáveis de controle pode auxiliar no tratamento de fatores que confundem a relação direta entre as variáveis educacionais e contagens de dispositivos por faixas de velocidade de acesso à Internet. Para tal, utilizaremos os [indicadores sociais](#Indicadores-sociais) que dizem respeito à **expectativa de vida**, **taxa de pobreza**, **taxa de analfabetismo**, **PIB per capita**, **mortalidade infantil** e **taxa de gravidez infanto-juvenil** dos municípios, e também consideraremos a **contagem de operadoras** para cada observação que compõem o dataset; estas serão chamadas **covariáveis**. Ademais, *dummies* para cada região geográfica também foram adicionadas a todos os modelos.

Nesta etapa, especificamos quatro modelos distintos:

* **Modelo *main***: adota a contagem de dispositivos/habitante com acesso à Internet de 0 a 2 Mb/s como principal variável explicativa.
* **Modelo de todas as faixas**: considera as variáveis de contagem de dispositivos/habitante para todas as faixas de velocidade como principais variáveis explicativas.
* **Modelo não linear**: considera a contagem de dispositivos/habitante com acesso à Internet de 0 a 2 Mb/s e seu quadrado – este último visa captar os retornos marginais da contagem de dispositivos – como principais variáveis explicativas.
* **Modelo de interação**: considera a contagem de dispositivos/habitante com acesso à Internet de 0 a 2 Mb/s e sua interação com a contagem de operadoras no município – mecanismo de captura de efeitos de competição – como principais variáveis explicativas.

Para tal, dividimos nossos dados entre grupos de treino, validação e teste, contendo, respectivamente, 80%, 10% e 10% do dataset. As observações que compõem cada grupo foram selecionadas aleatoriamente – porém, fixamos a semente responsável pelo sorteio em nossa biblioteca de implementações.

Para cada nível de ensino e especificação de modelo, experimentamos com as seguintes variáveis dependentes:

* Taxa de aprovação
* Nota de Matemática no SAEB
* Nota de Língua Portuguesa no SAEB
* IDEB

Os resultados e conclusões são apresentados a seguir.

### Modelo *main*

### Modelo de todas as faixas

### Modelo não linear

### Modelo de interação

# Conclusões

# Apêndices

## A. Limpeza e manipulação das bases brutas

## B. Gráficos de análise exploratória

### 1. Dados de Internet

Clique [aqui](#Conclus%C3%B5es---Internet) para retornar à leitura.

#### Distribuição de dispositivos/habitante por faixa de velocidade, por região

In [ ]:
exploratory_analysis.get_internet_plot(dataset, 'device_distribution')

#### Distribuição da média de operadoras, por região

In [ ]:
exploratory_analysis.get_internet_plot(dataset, 'isp_distribution')

### 2. Dados de Educação

Clique [aqui](#Conclus%C3%B5es---Educa%C3%A7%C3%A3o) para retornar à leitura.

#### IDEB

In [ ]:
exploratory_analysis.get_education_plot(dataset, 'ideb')

#### Taxas de aprovação

In [ ]:
exploratory_analysis.get_education_plot(dataset, 'approval_rate')

#### SAEB - Matemática

In [ ]:
exploratory_analysis.get_education_plot(dataset, 'saeb_mat')

#### SAEB - Língua Portuguesa

In [ ]:
exploratory_analysis.get_education_plot(dataset, 'saeb_port')

### 3. Indicadores Sociais

Clique [aqui](#Conclus%C3%B5es---Indicadores-sociais) para retornar à leitura.

#### População por região

In [ ]:
exploratory_analysis.get_social_plot(dataset, 'population')

#### Expectativa de vida

In [ ]:
exploratory_analysis.get_social_plot(dataset, 'life_expectancy')

#### Analfabetismo

In [ ]:
exploratory_analysis.get_social_plot(dataset, 'analfabetism')

#### Taxa de pobreza

In [ ]:
exploratory_analysis.get_social_plot(dataset, 'poverty')

#### PIB per capita

In [ ]:
exploratory_analysis.get_social_plot(dataset, 'gdp_per_capita')

#### Mortalidade infantil

In [ ]:
exploratory_analysis.get_social_plot(dataset, 'child_mortality')

#### Gravidez na adolescência

In [ ]:
exploratory_analysis.get_social_plot(dataset, 'adolescent_pregnancy')

## C. Testes de Hipótese

### 1. Estatística do Teste A/B: diferenças entre taxas de aprovação por estado

Clique [aqui](#Diferen%C3%A7as-entre-taxas-de-aprova%C3%A7%C3%A3o-por-regi%C3%A3o) para voltar à leitura.

#### Ensino Fundamental I

In [ ]:
hypothesis_tests.print_approval_rate_comparison(dataset, 'tx_aprov_ef_1')

#### Ensino Fundamental II

In [ ]:
hypothesis_tests.print_approval_rate_comparison(dataset, 'tx_aprov_ef_2')

#### Ensino Médio

In [ ]:
hypothesis_tests.print_approval_rate_comparison(dataset, 'tx_aprov_em')

### 2. Estatística do Teste A/B: diferenças entre taxas de aprovação entre municípios pobres e ricos

Clique [aqui](#Diferen%C3%A7as-entre-taxas-de-aprova%C3%A7%C3%A3o-de-munic%C3%ADpios-pobres-e-ricos) para voltar à leitura.

#### Visão geral

In [ ]:
hypothesis_tests.get_histogram(dataset, 'dist_per_capita_gdp')

#### Ensino Fundamental I

In [ ]:
hypothesis_tests.get_histogram(dataset, 'bootstrap_ef_1_gdp')

#### Ensino Fundamental II

In [ ]:
hypothesis_tests.get_histogram(dataset, 'bootstrap_ef_2_gdp')

#### Ensino Médio

In [ ]:
hypothesis_tests.get_histogram(dataset, 'bootstrap_em_gdp')

### 3. Estatística do Teste A/B: diferenças entre taxas de aprovação entre municípios com maior e menor acesso à Internet de alta velocidade

Clique [aqui](#Diferen%C3%A7as-entre-taxas-de-aprova%C3%A7%C3%A3o-entre-munic%C3%ADpios-com-menor-e-maior-acesso-%C3%A0-Internet-de-alta-velocidade) para voltar à leitura.

#### Visão geral

In [ ]:
hypothesis_tests.get_histogram(dataset, 'dist_internet_connection')

#### Ensino Fundamental I

In [ ]:
hypothesis_tests.get_histogram(dataset, 'bootstrap_ef_1_internet_connection')

#### Ensino Fundamental II

In [ ]:
linear_regression.get_histogram(dataset, 'bootstrap_ef_2_internet_connection')

#### Ensino Médio

In [ ]:
hypothesis_tests.get_histogram(dataset, 'bootstrap_em_internet_connection')

## D. Regressões

### 1. Gráficos - Regressões univariadas

Clique [aqui](#Regress%C3%B5es-univariadas) para voltar à leitura.

#### I. Dispositivos/hab na faixa 0 a 2 Mb/s e taxa de aprovação no Ensino Fundamental I

In [ ]:
linear_regression.get_univariate_regression_plot(dataset, 'ef_1_reg'), linear_regression.get_univariate_regression_plot(dataset, 'ef_1')

#### II. Dispositivos/hab na faixa 0 a 2 Mb/s e taxa de aprovação no Ensino Fundamental II

In [ ]:
linear_regression.get_univariate_regression_plot(dataset, 'ef_2_reg'), linear_regression.get_univariate_regression_plot(dataset, 'ef_2')

#### III. Dispositivos/hab na faixa 0 a 2 Mb/s e taxa de aprovação no Ensino Médio

In [ ]:
linear_regression.get_univariate_regression_plot(dataset, 'em_reg'), linear_regression.get_univariate_regression_plot(dataset, 'em')

### 2. Gráficos - Regressões multivariadas

Clique [aqui](#Regress%C3%B5es-multivariadas) para voltar à leitura.